In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current sessionadd Codeadd Markdown

In [2]:
# Install YOLOv8 dependencies
!pip install ultralytics
!pip install opencv-python-headless
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.3/905.3 kB 23.1 MB/s eta 0:00:0000:01


In [2]:
import os
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import torch
from tqdm import tqdm  # For progress bar


# Create YOLO format dataset
def create_yolo_format(dataset_dir, train_csv, image_dir):
    # Create necessary directories
    os.makedirs(f'{dataset_dir}/images/train', exist_ok=True)
    os.makedirs(f'{dataset_dir}/images/val', exist_ok=True)
    os.makedirs(f'{dataset_dir}/labels/train', exist_ok=True)
    os.makedirs(f'{dataset_dir}/labels/val', exist_ok=True)

    # Load the CSV file
    train_df = pd.read_csv(train_csv)

    # Split dataset into train and validation sets
    train_df, val_df = train_test_split(
        train_df, 
        test_size=0.3, 
        random_state=42, 
        stratify=train_df['class']  # Stratify by the class column
    )
    
    # Class mapping
    class_map = {
    'PNN': 0,
    'Lysee': 1,
    'LAM3': 2,
    'LLC': 3,
    'LyB': 4,
    'MO': 5,
    'MBL': 6,
    'LGL': 7,
    'EO': 8,
    'Thromb': 9,
    'Er': 10,
    'B': 11,
    'LF': 12,
    'LY': 13,
    'M': 14,
    'MM': 15,
    'LH_lyAct': 16,
    'MoB': 17,
    'PM': 18,
    'BA': 19,
    'LM': 20,
    'LZMG': 21,
    'SS': 22
    }


    # Convert bounding boxes to YOLO format
    def convert_to_yolo_format(image_shape, x1, y1, x2, y2, class_abbr):
        h, w = image_shape[:2]
        x_center = (x1 + x2) / 2 / w
        y_center = (y1 + y2) / 2 / h
        width = (x2 - x1) / w
        height = (y2 - y1) / h
        class_id = class_map.get(class_abbr, -1)
        if class_id == -1:
            return None  # Invalid class
        return f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"

    # Function to process and save images/labels
    def process_image(image_filename, x1, y1, x2, y2, class_abbr, set_type):
        image_path = os.path.join(image_dir, image_filename)

        # Skip if the image does not exist
        if not os.path.exists(image_path):
            return None, None

        # Read the image
        image = cv2.imread(image_path)
        if image is None:
            return None, None

        # Save image without augmentation
        dest_image_path = os.path.join(dataset_dir, 'images', set_type, image_filename)
        cv2.imwrite(dest_image_path, image)

        # Convert bounding box to YOLO format
        yolo_label = convert_to_yolo_format(image.shape, x1, y1, x2, y2, class_abbr)
        if yolo_label is None:
            return None, None
        
        label_filename = os.path.splitext(image_filename)[0] + '.txt'
        label_path = os.path.join(dataset_dir, 'labels', set_type, label_filename)

        # Save YOLO label
        with open(label_path, 'w') as f:
            f.write(yolo_label)
        
        return dest_image_path, label_path

    # Process train and validation datasets
    def process_dataset(df, set_type):
        for _, row in tqdm(df.iterrows(), total=df.shape[0], desc=f"Processing {set_type} dataset"):
            image_filename = row['NAME']
            x1, y1, x2, y2, class_abbr = row['x1'], row['y1'], row['x2'], row['y2'], row['class']
            process_image(image_filename, x1, y1, x2, y2, class_abbr, set_type)

    # Process both datasets
    process_dataset(train_df, 'train')
    process_dataset(val_df, 'val')


# Create data.yaml file for YOLO
def create_data_yaml(dataset_dir):
    data_yaml_content = f"""
train: {dataset_dir}/images/train
val: {dataset_dir}/images/val
nc: 23
names: ['PNN', 'Lysee', 'LAM3', 'LLC', 'LyB', 'MO', 'MBL', 'LGL', 'EO', 'Thromb', 'Er', 'B', 'LF', 'LY', 'M', 'MM', 'LH_lyAct', 'MoB', 'PM', 'BA', 'LM', 'LZMG', 'SS']
"""
    with open(f'{dataset_dir}/data.yaml', 'w') as f:
        f.write(data_yaml_content)



# Main script execution
if __name__ == "__main__":
    # Define paths
    dataset_dir = '/kaggle/working/dataset_cytologia'
    train_csv_path = '/kaggle/input/cytologia-dataset/cytologia-data-1732098640162.csv'
    image_dir = '/kaggle/input/cytologia-dataset/images(1)/dataset_cytologia'

    # Step 1: Create YOLO format dataset
    create_yolo_format(dataset_dir, train_csv_path, image_dir)

    # Step 2: Create data.yaml file
    create_data_yaml(dataset_dir)



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Processing val dataset: 100%|██████████| 15825/15825 [02:46<00:00, 94.95it/s] 


In [ ]:
# Train YOLOv8 model
def train_yolov11(dataset_dir):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = YOLO("yolov8n.pt")  # Load YOLOv8 pre-trained model
    model.train(
        data=f'{dataset_dir}/data.yaml',
        epochs=50,
        imgsz=640,
        batch=32,  # Adjusted batch size for typical GPU memory
        val=True  # Enable validation during training
    )

train_yolov11(dataset_dir)


100%|██████████| 6.25M/6.25M [00:00<00:00, 77.0MB/s]


Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/dataset_cytologia/data.yaml, epochs=50, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tru

100%|██████████| 755k/755k [00:00<00:00, 17.9MB/s]


Overriding model.yaml nc=80 with nc=23

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

100%|██████████| 5.35M/5.35M [00:00<00:00, 76.8MB/s]


...

      38/50      4.05G     0.3975      0.546     0.9289         54        640: 100%|██████████| 1081/1081 [05:42<00:00,  3.15it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 240/240 [01:14<00:00,  3.22it/s]


                   all      15323      15323      0.834      0.837      0.889      0.853

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 240/240 [01:14<00:00,  3.23it/s]


                   all      15323      15323      0.833      0.837      0.889      0.853

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      41/50      4.06G     0.3041      0.423     0.8594         32        640:  43%|████▎     | 464/1081 [02:28<03:17,  3.13it/s]

In [ ]:
import pandas as pd
import os
from ultralytics import YOLO

def infer_yolov11(model, test_csv, image_dir, output_csv, confidence_threshold=0.2):
    # Read test CSV to get the list of image filenames and trustii_id
    test_df = pd.read_csv(test_csv)
    trustii_ids = test_df['trustii_id'].tolist()  # Assuming trustii_id exists in the test CSV
    image_filenames = test_df['NAME'].tolist()  # Assuming the column with filenames is 'NAME'

    # Class name map (replace with actual class names from your model training)
    class_map = {
        0: 'PNN',
        1: 'Lysee',
        2: 'LAM3',
        3: 'LLC',
        4: 'LyB',
        5: 'MO',
        6: 'MBL',
        7: 'LGL',
        8: 'EO',
        9: 'Thromb',
        10: 'Er',
        11: 'B',
        12: 'LF',
        13: 'LY',
        14: 'M',
        15: 'MM',
        16: 'LH_lyAct',
        17: 'MoB',
        18: 'PM',
        19: 'BA',
        20: 'LM',
        21: 'LZMG',
        22: 'SS'
    }

    predictions = []

    for trustii_id, image_filename in zip(trustii_ids, image_filenames):
        # Construct the full path to the image
        image_path = os.path.join(image_dir, image_filename)

        if os.path.exists(image_path):
            # Perform inference
            results = model(image_path)

            # Ensure results are valid
            if isinstance(results, list):
                results = results[0]  # Get the first result if it's a list

            if hasattr(results, 'boxes') and len(results.boxes) > 0:
                boxes = results.boxes.xywh  # Bounding box coordinates (xywh format)
                scores = results.boxes.conf  # Confidence scores for each box
                class_ids = results.boxes.cls  # Class IDs for each box

                # Select the highest confidence prediction
                max_index = scores.argmax().item()  # Get index of max confidence
                if scores[max_index] >= confidence_threshold:
                    x_center, y_center, w, h = boxes[max_index][:4]
                    x1, y1 = x_center - w / 2, y_center - h / 2
                    x2, y2 = x_center + w / 2, y_center + h / 2
                    class_id = int(class_ids[max_index])  # Get class ID from cls attribute

                    # Get class name
                    class_name = class_map.get(class_id, 'Unknown')

                    predictions.append([trustii_id, image_filename, int(x1), int(y1), int(x2), int(y2), class_name])
                else:
                    predictions.append([trustii_id, image_filename, None, None, None, None, None])
            else:
                predictions.append([trustii_id, image_filename, None, None, None, None, None])
        else:
            print(f"Image {image_filename} not found in directory {image_dir}")
            predictions.append([trustii_id, image_filename, None, None, None, None, None])

    # Create dataframe for predictions
    df = pd.DataFrame(predictions, columns=['trustii_id', 'NAME', 'x1', 'y1', 'x2', 'y2', 'class'])

    # Ensure all rows from test_df are present
    result_df = test_df[['trustii_id', 'NAME']].merge(df, on=['trustii_id', 'NAME'], how='left')

    # Save to CSV
    result_df.to_csv(output_csv, index=False)

# Paths
test_csv_path = '/kaggle/input/cytologia-dataset/cytologia-data-1732098640162(1).csv'
image_dir_path = '/kaggle/input/cytologia-dataset/images(1)/dataset_cytologia'
output_csv_path = '/kaggle/working/predictions.csv'
model_path = '/kaggle/input/lastmodel/pytorch/default/1/best (8).pt'

# Load the trained YOLOv11 model
model = YOLO(model_path)

# Run inference
infer_yolov11(model, test_csv_path, image_dir_path, output_csv_path, confidence_threshold=0.2)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

image 1/1 /kaggle/input/cytologia-dataset/images(1)/dataset_cytologia/681daf42-3.jpg: 640x640 1 LLC, 9.8ms
Speed: 16.0ms preprocess, 9.8ms inference, 296.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/input/cytologia-dataset/images(1)/dataset_cytologia/172bf8a5-e.jpg: 640x640 1 Lysee, 8.4ms
Speed: 6.3ms preprocess, 8.4ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/input/cytologia-dataset/images(1)/dataset_cytologia/179a21ee-4.jpg: 640x640 1 M, 8.3ms
Speed: 4.0ms preprocess, 8.3ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/input/cytologia-dataset/images(1)/dataset_cytologia/f15